In [2]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split
import os
import numpy as np
import cv2

In [3]:
data_info = {
    "1": "Swing",
    "2": "Heel strike",
    "3": "Full contact",
    "4": "Heel off & toe off",
}

class Walk(object):
    def __init__(self, data_type:str, files: list):
        self.data_type = data_type
        self.files = files

walks = []

parent_path = "C:/python/dl2022/trainingset_ratio_50"
folders = os.listdir(parent_path)

for folder in folders:
    child_path = parent_path + "/" + folder
    files = os.listdir(child_path)
    walks.append(Walk(data_type=folder, files=files))


In [4]:
x = []
y = []

sigma = 3

for walk in walks:
    for file in walk.files:
        try:
            img = cv2.imread(parent_path + "/" + walk.data_type + "/" + file, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (28, 28))
            img = cv2.GaussianBlur(img, (0, 0), sigma)
            data = np.asarray(img)

            label = [0 for i in range(4)]
            label[int(walk.data_type) - 1] = 1

            x.append(img)
            y.append(label)
        except Exception as ex:
            print(ex)
            print("ERROR ON " + walk + " " + file)

X = np.array(x)
Y = np.array(y)

print(X.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=0.3)

print(x_train.shape)
print(y_train.shape)

        

(13635, 28, 28)
(13635, 4)
(9544, 28, 28)
(9544, 4)


In [5]:


model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu", input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model_dir = './model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
model_path = model_dir + "/dog_cat_classify.model"

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=7)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1605760   
                                                        

In [6]:
history = model.fit(x_train, y_train.astype(np.uint8), validation_split=0.2, epochs=3, batch_size=200, verbose=0, callbacks=[checkpoint, early_stopping])


Epoch 1: val_loss improved from inf to 0.83967, saving model to ./model\dog_cat_classify.model


INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets


INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets



Epoch 2: val_loss improved from 0.83967 to 0.69970, saving model to ./model\dog_cat_classify.model


INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets


INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets



Epoch 3: val_loss improved from 0.69970 to 0.49982, saving model to ./model\dog_cat_classify.model


INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets


INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets


In [7]:
print(history.history)

print("정확도 : %.4f" % (model.evaluate(x_test, y_test)[1]))

{'loss': [7.324771404266357, 0.8092600107192993, 0.6633427143096924], 'accuracy': [0.4998035430908203, 0.6666666865348816, 0.738572359085083], 'val_loss': [0.8396701812744141, 0.6996966600418091, 0.49982067942619324], 'val_accuracy': [0.6469355821609497, 0.7684651613235474, 0.8124672770500183]}
128/128 [==============================] - 1s 9ms/step - loss: 0.4776 - accuracy: 0.8299
정확도 : 0.8299
